In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
import plotly.graph_objects as go
from utils import preprocess_data_linear, create_dataset_linear
import pandas as pd
from plotly.subplots import make_subplots

# Favorita

In [24]:
# Leitura dos dados tratados
favorita = pd.read_parquet('Bases Tratadas/favorita_item_loja_semana.parquet')
favorita['Data'] = pd.to_datetime(favorita['Data'])

In [25]:
# Agrupamento de vendas por data
favorita_agrupado = favorita[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
favorita_agrupado.set_index('Data', inplace=True)

In [26]:
# Pré-processamento dos dados
train, test, scaler = preprocess_data_linear(favorita_agrupado['Vendas'].values, train_size_ratio=0.7)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento e teste
X_train, y_train = create_dataset_linear(train, look_back)
extended_test = np.concatenate([train[-look_back:], test])
X_test, y_test = create_dataset_linear(extended_test, look_back)

# Achatar os dados para o formato adequado ao modelo LinearRegression
X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

# Criar e treinar o modelo de Regressão Linear
start_train_time = time.time()
model = LinearRegression()
model.fit(X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = favorita_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = favorita_agrupado.index[:len(train)]

# Calcular o erro médio quadrático (RMSE)
print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()


Tempo de treino: 0.01907634735107422 segundos
Tempo de previsão: 0.000997304916381836 segundos


In [27]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Favorita/Tudo/RL.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})

# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Favorita/Tudo/RL.csv', index=False)

In [28]:
lojas_relevantes_favorita = [3, 8, 44, 45, 46, 47, 48, 49, 50, 51]
produtos_relevantes_favorita = {
    3: [1047679, 1503844],
    8: [1503844, 1047679],
    44: [1047679, 1503844],
    45: [1503844, 1473474],
    46: [584028, 1463992],
    47: [1503844, 1047679],
    48: [584028, 1463992],
    49: [1503844, 1473474],
    50: [584028, 364606],
    51: [1503844, 257847]
}

In [29]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_favorita])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_favorita):

    df_loja = favorita[favorita['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)

    train, test, scaler = preprocess_data_linear(df_loja_grouped['Vendas'].values, train_size_ratio=0.7)

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e teste
    X_train, y_train = create_dataset_linear(train, look_back)
    extended_test = np.concatenate([train[-look_back:], test])
    X_test, y_test = create_dataset_linear(extended_test, look_back)

    # Achatar os dados para o formato adequado ao modelo LinearRegression
    X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
    X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

    # Criar e treinar o modelo de Regressão Linear
    start_train_time = time.time()
    model = LinearRegression()
    model.fit(X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
    train_index = df_loja_grouped.index[:len(train)]

    # Adicionar resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })])

    # Plotando os dados originais
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=(i // 2) + 1, col=(i % 2) + 1)

    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

    # Salvar modelos e tempos
    modelos_por_loja[loja] = model
    tempos_treinamento[loja] = train_time
    tempos_previsao[loja] = forecast_time

# Add titles and labels
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Show the figure
fig.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja.to_csv('Resultados/Previsao/Favorita/Loja/RL.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Favorita/Loja/RL.csv', index=False)


In [30]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_favorita:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_favorita.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = favorita[(favorita['Loja'] == loja) & (favorita['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item_grouped.set_index('Data', inplace=True)

        # Pré-processamento dos dados
        train, test, scaler = preprocess_data_linear(df_loja_item_grouped['Vendas'].values, train_size_ratio=0.7)

        # Criar conjuntos de treinamento e teste
        X_train, y_train = create_dataset_linear(train, look_back)
        extended_test = np.concatenate([train[-look_back:], test])
        X_test, y_test = create_dataset_linear(extended_test, look_back)

        # Achatar os dados para o formato adequado ao modelo LinearRegression
        X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
        X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

        # Criar e treinar o modelo de Regressão Linear
        start_train_time = time.time()
        model = LinearRegression()
        model.fit(X_train, y_train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time

        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Garantir que os índices estão corretos
        test_index = df_loja_item_grouped.index[-len(y_test):]
        train_index = df_loja_item_grouped.index[:len(train)]

        # Adicionar resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })])

        # Plotando os dados originais
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

        # Salvar modelos e tempos
        modelos_por_loja_item[(loja, item)] = model
        tempos_treinamento[(loja, item)] = train_time
        tempos_previsao[(loja, item)] = forecast_time

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
    # Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item.to_csv('Resultados\Previsao\Favorita\Item\RL.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados\Tempo\Favorita\Item\RL.csv', index=False)
    

# M5

In [31]:
file_path_m5 = 'Bases Tratadas/m5_item_loja_semana.parquet'
df_m5 = pd.read_parquet(file_path_m5)
display(df_m5.head())

,Produto,Loja,Data,Vendas
0,HOBBIES_1_001,CA_1,1,0
1,HOBBIES_1_002,CA_1,1,0
2,HOBBIES_1_003,CA_1,1,0
3,HOBBIES_1_004,CA_1,1,0
4,HOBBIES_1_005,CA_1,1,0


In [32]:
# Agrupamento de vendas por data
m5_agrupado = df_m5[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
m5_agrupado.set_index('Data', inplace=True)

display(m5_agrupado)

,Vendas
Data,
1,161932
2,194735
3,185293
4,170194
5,170777
...,...
273,287796
274,282731
275,314216


In [33]:
# Pré-processamento dos dados
train, test, scaler = preprocess_data_linear(m5_agrupado['Vendas'].values, train_size_ratio=0.7)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento e teste
X_train, y_train = create_dataset_linear(train, look_back)
extended_test = np.concatenate([train[-look_back:], test])
X_test, y_test = create_dataset_linear(extended_test, look_back)

# Achatar os dados para o formato adequado ao modelo LinearRegression
X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

# Criar e treinar o modelo de Regressão Linear
start_train_time = time.time()
model = LinearRegression()
model.fit(X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = m5_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = m5_agrupado.index[:len(train)]

# Calcular o erro médio quadrático (RMSE)
print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()

Tempo de treino: 0.002645254135131836 segundos
Tempo de previsão: 0.0010867118835449219 segundos


In [34]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado_m5 = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado_m5.to_csv('Resultados/Previsao/M5/Tudo/RL.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo_m5 = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})

# Salvar o dataframe em um arquivo CSV
df_tempo_m5.to_csv('Resultados/Tempo/M5/Tudo/RL.csv', index=False)

In [35]:
lojas_relevantes_m5 = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

produtos_relevantes_m5 = {
    'CA_1': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_2': ['FOODS_3_586', 'FOODS_3_252'],
    'CA_3': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_4': ['FOODS_3_090', 'FOODS_3_586'],
    'TX_1': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_2': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_3': ['FOODS_3_586', 'FOODS_3_090'],
    'WI_1': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_2': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_3': ['FOODS_3_090', 'FOODS_3_226']
}

In [36]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_m5])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_m5):

    df_loja = df_m5[df_m5['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)

    train, test, scaler = preprocess_data_linear(df_loja_grouped['Vendas'].values, train_size_ratio=0.7)

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e teste
    X_train, y_train = create_dataset_linear(train, look_back)
    extended_test = np.concatenate([train[-look_back:], test])
    X_test, y_test = create_dataset_linear(extended_test, look_back)

    # Achatar os dados para o formato adequado ao modelo LinearRegression
    X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
    X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

    # Criar e treinar o modelo de Regressão Linear
    start_train_time = time.time()
    model = LinearRegression()
    model.fit(X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
    train_index = df_loja_grouped.index[:len(train)]

    # Adicionar resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })])

    # Plotando os dados originais
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=(i // 2) + 1, col=(i % 2) + 1)

    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

    # Salvar modelos e tempos
    modelos_por_loja[loja] = model
    tempos_treinamento[loja] = train_time
    tempos_previsao[loja] = forecast_time

# Add titles and labels
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Show the figure
fig.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja.to_csv('Resultados/Previsao/M5/Loja/RL.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/M5/Loja/RL.csv', index=False)

In [37]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_m5:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_m5.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = df_m5[(df_m5['Loja'] == loja) & (df_m5['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item_grouped.set_index('Data', inplace=True)

        # Pré-processamento dos dados
        train, test, scaler = preprocess_data_linear(df_loja_item_grouped['Vendas'].values, train_size_ratio=0.7)

        # Criar conjuntos de treinamento e teste
        X_train, y_train = create_dataset_linear(train, look_back)
        extended_test = np.concatenate([train[-look_back:], test])
        X_test, y_test = create_dataset_linear(extended_test, look_back)

        # Achatar os dados para o formato adequado ao modelo LinearRegression
        X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
        X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

        # Criar e treinar o modelo de Regressão Linear
        start_train_time = time.time()
        model = LinearRegression()
        model.fit(X_train, y_train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time

        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Garantir que os índices estão corretos
        test_index = df_loja_item_grouped.index[-len(y_test):]
        train_index = df_loja_item_grouped.index[:len(train)]

        # Adicionar resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })])

        # Plotando os dados originais
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

        # Salvar modelos e tempos
        modelos_por_loja_item[(loja, item)] = model
        tempos_treinamento[(loja, item)] = train_time
        tempos_previsao[(loja, item)] = forecast_time

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
# Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/M5/Item/RL.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/M5/Item/RL.csv', index=False)

# Olist

In [2]:
file_path_olist = 'Bases Tratadas/olist_item_loja_semana.parquet'
df_olist = pd.read_parquet(file_path_olist)
display(df_olist.head())

,Produto,Loja,Data,Vendas
0,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
1,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
2,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-05,1
3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19,3
4,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,2016-10-03,1


In [3]:
# Agrupamento de vendas por data
olist_agrupado = df_olist[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
olist_agrupado.set_index('Data', inplace=True)
display(olist_agrupado)

,Vendas
Data,
2016-09-05,3
2016-09-19,3
2016-10-03,9
2016-10-10,354
2016-12-26,1
...,...
2018-07-30,1865
2018-08-06,2423
2018-08-13,2145


In [4]:
lojas_relevantes_olist = [
    '6560211a19b47992c3666cc44a7e94c0', '4a3ca9315b744ce9f8e9374361493884', 
    '1f50f920176fa81dab994f9023523100', 'cc419e0650a3c5ba77189a1882b7556a', 
    'da8622b14eb17ae2831f4ac5b9dab84a', '955fee9216a65b617aa5c0531780ce60', 
    '1025f0e2d44d7041d6cf58b6550e0bfa', '7c67e1448b00f6e969d365cea6b010ab', 
    'ea8482cd71df3c1969d7b9473ff13abc', '7a67c85e85bb2ce8582c35f2203ad736'
]

produtos_relevantes_olist = {
    '6560211a19b47992c3666cc44a7e94c0': ['a92930c327948861c015c919a0bcb4a8', 'd017a2151d543a9885604dc62a3d9dcc'],
    '4a3ca9315b744ce9f8e9374361493884': ['99a4788cb24856965c36a24e339b6058', '35afc973633aaeb6b877ff57b2793310'],
    '1f50f920176fa81dab994f9023523100': ['422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b'],
    'cc419e0650a3c5ba77189a1882b7556a': ['154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'],
    'da8622b14eb17ae2831f4ac5b9dab84a': ['e0cf79767c5b016251fe139915c59a26', 'fc1d8637c0268af3db482c14b7ef8e75'],
    '955fee9216a65b617aa5c0531780ce60': ['aca2eb7d00ea1a7b8ebd4e68314663af', '54d9ac713e253fa1fae9c8003b011c2a'],
    '1025f0e2d44d7041d6cf58b6550e0bfa': ['b532349fe46b38fbc7bb3914c1bdae07', '9ecadb84c81da840dbf3564378b586e9'],
    '7c67e1448b00f6e969d365cea6b010ab': ['8ed094bfe076c568f6bb10feada3f75d', '3eef0cb94ba82de806bb30ab743c7655'],
    'ea8482cd71df3c1969d7b9473ff13abc': ['97017430754804328eb9597b7f85da03', 'c1f5307decb89342351bec53668cffd9'],
    '7a67c85e85bb2ce8582c35f2203ad736': ['c6dd917a0be2a704582055949915ab32', '601a360bd2a916ecef0e88de72a6531a']
}

In [5]:
# Pré-processamento dos dados
train, test, scaler = preprocess_data_linear(olist_agrupado['Vendas'].values, train_size_ratio=0.7)

# Definir o look_back
look_back = 10

# Criar conjuntos de treinamento e teste
X_train, y_train = create_dataset_linear(train, look_back)
extended_test = np.concatenate([train[-look_back:], test])
X_test, y_test = create_dataset_linear(extended_test, look_back)

# Achatar os dados para o formato adequado ao modelo LinearRegression
X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

# Criar e treinar o modelo de Regressão Linear
start_train_time = time.time()
model = LinearRegression()
model.fit(X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = model.predict(X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Garantir que os índices estão corretos
test_index = olist_agrupado.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
train_index = olist_agrupado.index[:len(train)]

# Calcular o erro médio quadrático (RMSE)
print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()

Tempo de treino: 0.0042572021484375 segundos
Tempo de previsão: 0.0 segundos


In [6]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado_olist = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado_olist.to_csv('Resultados/Previsao/Olist/Tudo/RL.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo_olist = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})

# Salvar o dataframe em um arquivo CSV
df_tempo_olist.to_csv('Resultados/Tempo/Olist/Tudo/RL.csv', index=False)

In [7]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_olist])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_olist):

    df_loja = df_olist[df_olist['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()
    date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
    date_range = date_range + pd.DateOffset(days=1)
    df_loja_grouped = df_loja_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
    df_loja_grouped.set_index('Data', inplace=True)

    train, test, scaler = preprocess_data_linear(df_loja_grouped['Vendas'].values, train_size_ratio=0.7)

    # Definir o look_back
    look_back = 10

    # Criar conjuntos de treinamento e teste
    X_train, y_train = create_dataset_linear(train, look_back)
    extended_test = np.concatenate([train[-look_back:], test])
    X_test, y_test = create_dataset_linear(extended_test, look_back)

    # Achatar os dados para o formato adequado ao modelo LinearRegression
    X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
    X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

    # Criar e treinar o modelo de Regressão Linear
    start_train_time = time.time()
    model = LinearRegression()
    model.fit(X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time

    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[-len(y_test):]  # Ajustado para corresponder ao conjunto de teste estendido
    train_index = df_loja_grouped.index[:len(train)]

    # Adicionar resultados ao dataframe
    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })])

    # Plotando os dados originais
    fig.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=(i // 2) + 1, col=(i % 2) + 1)
    fig.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=(i // 2) + 1, col=(i % 2) + 1)

    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

    # Salvar modelos e tempos
    modelos_por_loja[loja] = model
    tempos_treinamento[loja] = train_time
    tempos_previsao[loja] = forecast_time

# Add titles and labels
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Show the figure
fig.show()

# Salvar os dataframes em arquivos CSV
df_resultado_loja.to_csv('Resultados/Previsao/Olist/Loja/RL.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Olist/Loja/RL.csv', index=False)

In [8]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_olist.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = df_olist[(df_olist['Loja'] == loja) & (df_olist['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        try:
            df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
            date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
            date_range = date_range + pd.DateOffset(days=1)
            df_loja_item_grouped = df_loja_item_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
            df_loja_item_grouped.set_index('Data', inplace=True)

            # Pré-processamento dos dados
            train, test, scaler = preprocess_data_linear(df_loja_item_grouped['Vendas'].values, train_size_ratio=0.7)

            # Criar conjuntos de treinamento e teste
            X_train, y_train = create_dataset_linear(train, look_back)
            extended_test = np.concatenate([train[-look_back:], test])
            X_test, y_test = create_dataset_linear(extended_test, look_back)

            # Achatar os dados para o formato adequado ao modelo LinearRegression
            X_train = X_train.reshape(X_train.shape[0], -1)  # (n_amostras, look_back)
            X_test = X_test.reshape(X_test.shape[0], -1)    # (n_amostras, look_back)

            # Criar e treinar o modelo de Regressão Linear
            start_train_time = time.time()
            model = LinearRegression()
            model.fit(X_train, y_train)
            end_train_time = time.time()
            train_time = end_train_time - start_train_time

            # Realizar previsões no conjunto de teste
            start_forecast_time = time.time()
            test_predict = model.predict(X_test)
            end_forecast_time = time.time()
            forecast_time = end_forecast_time - start_forecast_time

            # Reverter a normalização das previsões e dos valores reais
            test_predict_original = scaler.inverse_transform(test_predict.reshape(-1, 1)).flatten()
            y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

            # Garantir que os índices estão corretos
            test_index = df_loja_item_grouped.index[-len(y_test):]
            train_index = df_loja_item_grouped.index[:len(train)]

            # Adicionar resultados ao dataframe
            df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
                'Data': test_index,
                'Loja': loja,
                'Item': item,
                'Real': y_test_original,
                'Previsto': test_predict_original
            })])

            df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
                'Loja': [loja],
                'Item': [item],
                'Treino': [train_time],
                'Previsao': [forecast_time]
            })])

            # Plotando os dados originais
            fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=train_index, y=scaler.inverse_transform(train).flatten(), mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

            fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
            fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

            # Salvar modelos e tempos
            modelos_por_loja_item[(loja, item)] = model
            tempos_treinamento[(loja, item)] = train_time
            tempos_previsao[(loja, item)] = forecast_time
        except:
            continue

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
# Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Olist/Item/RL.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Olist/Item/RL.csv', index=False)